In [1]:
#%load_ext autoreload
#%autoreload 2

import torch
from torch.utils.data import DataLoader
from dataset import RecoveryRateDataset
import os
from scaling_utils import compute_mean_and_std, apply_std_scaling
from model import LSTMEstimator
from training_loop import model_train
import wandb

In [2]:
import wandb

wandb.init(project="my-test-project", entity="aksoym")

wandb: Currently logged in as: aksoym (use `wandb login --relogin` to force relogin)


In [3]:
BATCH_SIZE = 32
LR = 0.001
N_EPOCHS = 1000

In [4]:
wandb.config = {
  "learning_rate": LR,
  "epochs": N_EPOCHS,
  "batch_size": BATCH_SIZE
}

In [5]:
root_dir = os.getcwd().split('Start_PyCharm')[0]
data_file_path = root_dir + 'Start_PyCharm/state_vector_dataframes_as_training_data/data/airport_state_df_w_features.pickle'


SEQUENCE_LENGTH = 5

features_to_drop = ['reg_type', 'reg_bool_type', 'reg_cause']
rr_dataset = RecoveryRateDataset(data_file_path, features_to_drop=features_to_drop, sequence_length=SEQUENCE_LENGTH,
                                 fill_with='backfill')


train_val_test_ratios = (0.6, 0.2, 0.2)
train, val, test = torch.utils.data.random_split(rr_dataset, lengths=[int(len(rr_dataset)*ratio)
                                                                      for ratio in train_val_test_ratios])

#Compute mean and std for standard scaling.
training_mean, training_std = compute_mean_and_std(train)

scaled_training_data = apply_std_scaling(train, training_mean, training_std)
scaled_val_data = apply_std_scaling(val, training_mean, training_std)
scaled_test_data = apply_std_scaling(test, training_mean, training_std)


#Training parameters.

training_data_loader = DataLoader(scaled_training_data, batch_size=BATCH_SIZE, shuffle=True)4
val_data_loader = DataLoader(scaled_val_data, batch_size=128, shuffle=True)
test_data_loader = DataLoader(scaled_test_data)

In [ ]:
model = LSTMEstimator(len(rr_dataset.feature_names), initial_dense_layer_size=16, dense_parameter_multiplier=2,
                      dense_layer_count=3, lstm_layer_count=3, lstm_hidden_units=32, 
                      sequence_length=SEQUENCE_LENGTH).to(torch.device('cuda:0'))

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
lr_scheduler = None # Optional
loss_func = torch.nn.MSELoss()


model_save_path='best_model'

train_history = []
val_history = []

model_train(model, optimizer, loss_func, N_EPOCHS, training_data_loader, val_data_loader, train_history, val_history, model_save_path)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2167/2167 [00:52<00:00, 41.30it/s]



 Starting epoch 2, loss: 0.804165862902937, val_loss: 0.7177061000581604 lr= 0.001

 Validation loss inf --> 0.7177061000581604. Saving best model.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2167/2167 [00:49<00:00, 44.11it/s]



 Starting epoch 3, loss: 0.7102848948815422, val_loss: 0.7162428728783328 lr= 0.001

 Validation loss 0.7177061000581604 --> 0.7162428728783328. Saving best model.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2167/2167 [00:50<00:00, 43.04it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2167/2167 [00:23<00:00, 92.37it/s]



 Starting epoch 343, loss: 0.4113408098290414, val_loss: 0.6366105343920091 lr= 0.001


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2167/2167 [00:23<00:00, 91.55it/s]



 Starting epoch 344, loss: 0.416107568764362, val_loss: 0.65678270498692 lr= 0.001


  3%|████▉                                                                                                                                                                                          | 56/2167 [00:00<00:16, 130.88it/s]